Theoretical part

1a) P($m^{th}$ experiment gives significant result | m experiments lacking power to reject $H_0$)

This probability of an experiment lacking the power to reject $H_0$ is ($1 -\alpha  $). Assuming that consecutive experiments are  independent, the probability that the $m^{th}$ experiment gives significant result is: 

$(1-\alpha)^{m-1} \cdot \alpha $ OR $ \alpha$

TODO: KIEZEN WELKE VERSIE VAN HET ANTWOORD HIERBOVEN WE INLEVEREN

1b) P(at least one significant result | m experiments lacking power to reject $H_0$)

This probability is given by: 1$-$ P(no significant results | m experiments lacking power to reject $H_0$) +
This then equals:

$
1 - (1 - \alpha  )^{m}  
$

2) 

Suppose we have the following two lists that we want to interleave.

RankA
1 2 4

RankB
2 3 4

Suppose 1 and 2 are clicked on 48% of the times, and 3 is clicked on 4% of the times. This means RankA gives us 96% satisfaction and RankB gives us 52% satisfaction, so RankA is the best algorithm.

The coin tosses and interleaved lists are given below

$\textbf{AA}$

1A 2B 4A (A wins 50% of the time)


$\textbf{AB}$

1A 2B 3B (A wins 48% of the time)


$\textbf{BA}$

2B 1A 4A (A wins 50% of the time)


$\textbf{BB}$

2B 1A 3B (A wins 48% of the time)

This means that the team draft interleaving is unfair to the better algorithm.


Experimental part

1) Step 1: Simulate Rankings of Relevance for E and P (5 points)

In the next section, we first create a list of all combinations of relevances. We use itertools.product which gives all possible combinations of a list in any order. Then we use permutation which gives all the combinations of experiment and production relevances. We use itertools.permutations to do this.

In [111]:
import itertools
values = ['N','R','HR'] #possible values of a prediction

relevances = [] #relevances contains all combinations of N/R/HR with length 5
for r in itertools.product(values, repeat=5):
    relevances.append(list(r))

In [112]:
combinations = [] #combinations contains all pairs of relevances
for p in itertools.permutations(relevances, 2):
    combinations.append(list(p)) #we use this to get rid of the permutations object

In [113]:
print(combinations[:10]) #show the first 10 combinations

[[['N', 'N', 'N', 'N', 'N'], ['N', 'N', 'N', 'N', 'R']], [['N', 'N', 'N', 'N', 'N'], ['N', 'N', 'N', 'N', 'HR']], [['N', 'N', 'N', 'N', 'N'], ['N', 'N', 'N', 'R', 'N']], [['N', 'N', 'N', 'N', 'N'], ['N', 'N', 'N', 'R', 'R']], [['N', 'N', 'N', 'N', 'N'], ['N', 'N', 'N', 'R', 'HR']], [['N', 'N', 'N', 'N', 'N'], ['N', 'N', 'N', 'HR', 'N']], [['N', 'N', 'N', 'N', 'N'], ['N', 'N', 'N', 'HR', 'R']], [['N', 'N', 'N', 'N', 'N'], ['N', 'N', 'N', 'HR', 'HR']], [['N', 'N', 'N', 'N', 'N'], ['N', 'N', 'R', 'N', 'N']], [['N', 'N', 'N', 'N', 'N'], ['N', 'N', 'R', 'N', 'R']]]


2) Step 2: Implement Evaluation Measures (10 points)

In the next section we take two assumptions:

1) Values for the prediction relevances are N=0, R=1, HR=2

2) The amount of relevant predictions (overall) is assumed to be the total amount of relevant (R or HR) docs in the prediction set. So we assume there is no overlap between articles in the prediction.

In [114]:
#the first binary evaluation methods: average precision
numeric_map = {'N':0, 'R':1, 'HR':2} #we use this numeric map to map N/R/HR to a numeric value.
prediction = ['R','HR','N','R','N'] #this is a sample prediction to test functions

def count_rel(prediction1,prediction2):
    return sum(1 for i in prediction1 if i != 'N') + sum(1 for i in prediction2 if i != 'N')

def average_precision(prediction, r):
    ap = 0
    relevant_preds = 0
    for i in range(0,len(prediction)):
        if prediction[i] != 'N':
            relevant_preds += 1
            ap += relevant_preds/(i+1)
    return ap/r

ap = average_precision(prediction, count_rel(prediction, prediction))
print(ap)

0.4583333333333333


Now we will implement both multi-graded evaluation methods. 

The first is nDCG@k which requires a optimal prediction to normalize predictions. Here we will use the total amount of HR/R files to create an optimum prediction. Again we assume there is no overlap in predictions.

The second one is ERR, this model does not need any assumptions.

In [115]:
#nDCG@K
import numpy as np #Numpy is amazing right?

def generate_opt(prediction1, prediction2): #generate optimal sequences from two predictions
    opt_pred = []
    num_hr = sum(1 for i in prediction1 if i == 'HR') + sum(1 for i in prediction2 if i == 'HR')
    num_r  = sum(1 for i in prediction1 if i == 'R') + sum(1 for i in prediction2 if i == 'R')
    for i in range(min(num_hr,5)): #check if num_hr exceeds 5, fill with HR's
        opt_pred.append('HR')
    for i in range(min(5-num_hr,num_r)): #check if num_r exceeds the space left, will with R's
        opt_pred.append('R')
    for i in range(5-len(opt_pred)): #fill the rest with N
        opt_pred.append('N')
    return opt_pred

def dcg_k(numeric_map, prediction, opt_pred, k):
    dcg_opt = 0
    dcg = 0
    for i in range(0,k): #for the range until K, we sum both the optimum and prediction dcg
        dcg_opt += (2**numeric_map[opt_pred[i]]-1)/np.log2(1+i+1)
        dcg +=(2**numeric_map[prediction[i]]-1)/np.log2(1+i+1)
    return dcg/dcg_opt #dcg is normalized compared to the optimum
ndcg = dcg_k(numeric_map, prediction, generate_opt(prediction,prediction), 3)
print(ndcg)

0.53641800576


In [116]:
#ERR
def ERR(numeric_map, prediction):
    err = 0
    max_val = 2**max(list(numeric_map.values()))
    thetas = [(2**numeric_map[p]-1)/max_val for p in prediction]
    for i in range(0,len(prediction)):
        prod_val = 1
        for j in range(0,i):
            prod_val *= (1-thetas[j])*thetas[i]
        prod_val *= 1/(i+1)
        err += prod_val
    return err
err = ERR(numeric_map, prediction)
print(err)

1.281982421875


3) Step 3: Calculate the 𝛥measure (0 points)

In [117]:
#Here we calculate the delta measure for the three offline performance measures.
import math
def check_performance(combinations):
    ap_counter = 0
    ap_tie_counter = 0
    ap_delta = []
    err_counter = 0
    err_tie_counter = 0
    err_delta = []
    ndcg_counter = 0
    ndcg_tie_counter = 0
    ndcg_delta = []
    
    total_count = len(combinations)
    k = 5
    for s in combinations:
      
        prediction_e = s[0]
        prediction_p = s[1]

        
        r = count_rel(prediction_e, prediction_p)
        ap_e, ap_p = average_precision(prediction_e, r), average_precision(prediction_p, r)
        if ap_e > ap_p:
            ap_counter += 1
            ap_delta.append(ap_e-ap_p)
        if ap_e == ap_p:
            ap_tie_counter += 1
            
        ERR_e, ERR_p = ERR(numeric_map, prediction_e), ERR(numeric_map, prediction_p)
        if ERR_e > ERR_p:
            err_counter += 1
            err_delta.append(ERR_e-ERR_p)
        if ERR_e == ERR_p:
            err_tie_counter += 1
        opt_prediction = generate_opt(prediction_e,prediction_p)
        ndcg_e = dcg_k(numeric_map, prediction_e, opt_prediction, k)
        ndcg_p = dcg_k(numeric_map, prediction_p, opt_prediction, k)
        if ndcg_e > ndcg_p:
            ndcg_counter += 1
            ndcg_delta.append(ndcg_e-ndcg_p)
        if ndcg_e == ndcg_p:
            ndcg_tie_counter += 1
        
    print("Percentage of cases that E outperforms P with average precision measure:",round((ap_counter/total_count),2))
    print("Percentage of cases that E outperforms P with ERR measure:",round((err_counter/total_count),2))
    print("Percentage of cases that E outperforms P with NDCG measure:",round((ndcg_counter/total_count),2))
    print("Percentage of cases that E ties with P with average precision measure:",round((ap_tie_counter/total_count),2))
    print("Percentage of cases that E ties with P with ERR measure:",round((err_tie_counter/total_count),2))
    print("Percentage of cases that E ties with P with NDCG measure:",round((ndcg_tie_counter/total_count),2))
    return ap_delta,err_delta,ndcg_delta

The results of the delta measure show that for all three offline performance measures we see in 50% of the cases that the experiment ranking outperforms the production ranking. This is in line with expectations, since the combinations (rankings) are random and therefore we would expect that in a little less than 50% of the cases the experiment outperforms the production (since we also have ties its slightly less than 50%).

Step 4: Implement Interleaving (15 points)

In [129]:
import copy

def get_A_first(): #This is a function that determines is ranking A goes first or not
    A = np.random.uniform() # Take a random uniform number between 0 and 1    
    if A > 0.5: 
        return True
    else: 
        return False
    
def balanced_interleaving(s, cutoff = False):
    
    ranking_A = s[0]
    ranking_B = s[1]
    
    # Initialize
    I = []
    k_a, k_b = 0,0
        
    A_first = get_A_first() #Find out if A or B goes first
    
    # We assume that rankA and rankB contain 10 unique documents
    # That is why we can cast rankA and rankB to a dict
    # This makes it easier to return a list of length 9, while adhering to the pseudo code from the slides
    
    rankA = {}
    rankB = {}
    
    A = [i for i in range(0,5)]
    B = [i for i in range(5,10)]
    
    for i in A:
        rankA[i] = ranking_A[i]
        
    for j in B:
        rankB[j] = ranking_B[j-5] 
        
    # This code just follows the pseudo code from the slides
    while k_a+1 <= len(ranking_A) and k_b+1 <= len(ranking_B):
        if (k_a < k_b) or ((k_a == k_b) and A_first):
            if A[k_a] not in I:
                I.append(A[k_a])
            k_a += 1
            
        else:
            if B[k_b] not in I:
                I.append(B[k_b])
            k_b += 1
             
    # I is now filled with unique indices, we now have to convert these back to labels
    I_ids = copy.copy(I)
    for i in range(0,len(I)):
        try:
            I[i] = rankA[I[i]]
        except:
            I[i] = rankB[I[i]]
    
    if cutoff: #Cut-off to make the lenght of the list the same length as A and B
        return I[:len(ranking_A)], I_ids[:len(ranking_A)], ranking_A, ranking_B
    else:
        return I, I_ids, ranking_A, ranking_B
    
def team_based_interleaving(s, cutoff = False):
    
    ranking_A = s[0]
    ranking_B = s[1]
    
    # Initialize
    I = []
    k_a, k_b = 0,0
    
    # We assume that rankA and rankB contain 10 unique documents
    # That is why we can cast rankA and rankB to a dict
    # This makes it easier to return a list of length 9, while adhering to the pseudo code from the slides
    
    rankA = {}
    rankB = {}
    
    A = [i for i in range(0,5)]
    B = [i for i in range(5,10)]
    
    for i in A:
        rankA[i] = ranking_A[i]
        
    for j in B:
        rankB[j] = ranking_B[j-5] 
        
    # This code just follows the pseudo code from the slides
    teamA = 0
    teamB = 0
    
    for i in range(0,len(ranking_A)):
        A_first = get_A_first() # Flip a coin each time we go again
        if A_first:
            for idx in A: #Loop over A
                if idx not in I: #Top result in A not yet in I
                    I.append(idx)
                    break
            for idx in B: #Loop over B
                if idx not in I: #Top result in B not yet in I
                    I.append(idx)
                    break
        else:
            for idx in B: #Loop over B
                if idx not in I: #Top result in B not yet in I
                    I.append(idx)
                    break
            for idx in A: #Loop over A
                if idx not in I: #Top result in A not yet in I
                    I.append(idx)
                    break
                
    # I is now filled with unique indices, we now have to convert these back to labels
    I_ids = copy.copy(I)
    for i in range(0,len(I)):
        try:
            I[i] = rankA[I[i]]
        except:
            I[i] = rankB[I[i]]
    
    if cutoff: #Cut-off to make the lenght of the list the same length as A and B
        return I[:len(ranking_A)], I_ids[:len(ranking_A)], ranking_A, ranking_B
    else:
        return I, I_ids, ranking_A, ranking_B

def define_winner(clicks,I_ids):
            
    clicks_A = 0 # Number of clicks from result A
    clicks_B = 0 # Number or clicks from result B
    
    A = [i for i in range(0,5)]
    B = [i for i in range(5,10)]        
            
    for click in range(0,len(clicks)): # Loop over the clicks
        if clicks[click]:
            if I_ids[click] in A:
                clicks_A += 1
            elif I_ids[click] in B:
                clicks_B += 1
            
    if clicks_A > clicks_B:
        return "A"
    elif clicks_B > clicks_A:
        return "B"
    else: 
        return "Tie"
            
test_set = combinations[12347]
clicks = [True,True,False,False,False]

# I, I_ids, rank_A, rank_B = balanced_interleaving(test_set, True)
I, I_ids, rank_A, rank_B = team_based_interleaving(test_set, True)
print ('I is',I)

define_winner(clicks,I_ids)

A_first is  False
A_first is  True
A_first is  True
A_first is  False
A_first is  True
ranking_A is ['N', 'R', 'HR', 'HR', 'N']
ranking_B is ['N', 'N', 'N', 'R', 'HR']
I is ['N', 'N', 'R', 'N', 'HR']


'Tie'

 Step 5: Implement User Clicks Simulation (15 points)

In [119]:
#read search query data
import csv
import re

def read_data():

    answers = []
    query_ids = []
    clicks = []
    click = []
    last_type = 'C'

    with open('YandexRelPredChallenge.txt') as f:
        for line in f:
            vals = re.split(r'\t+', line.rstrip())
            line_type = vals[2] #we look at the type of data line
            if line_type == 'Q': #if type is query, we append the query.
                if len(click) > 0: #we append clicks of last query before we go further
                    clicks.append(click)
                    click = []
                answers.append(list(map(int, vals[5:])))
                query_ids.append(int(vals[3]))
            if last_type == 'Q' and line_type == 'Q': #If last type also was query there are no clicks  
                clicks.append([])
            elif line_type == 'C':
                click.append(int(vals[3]))
            last_type = vals[2]
        clicks.append(click) #we shall not forget the last click sequence...
    
    return answers,query_ids,clicks

answers,query_ids,clicks = read_data()

print('Example query, results and clicks:')
print("Query ID:", query_ids[5])
print("Query results:", answers[5])
print("Query clicks:", clicks[5])
print('We have ',len(clicks),' answers/click sequences in total!')


Example query, results and clicks:
Query ID: 9
Query results: [13, 70, 66, 94, 50, 104, 29, 21, 89, 85]
Query clicks: [104, 21]
We have  42652  answers/click sequences in total!


In [110]:
from collections import Counter

#Function to determine rho parameter of random click model given set of documents and clicks
def rcm(documents,clicks):
    unique_docs = []
    unique_clicks = []
    
    assert(len(documents) == len(clicks))
    
    #First we find all unique documents to get the count
    for d in documents:
        for e in d:
            unique_docs.append(e)

    #number_of_docs = len(Counter(unique_docs).keys())
    number_of_docs = len(unique_docs)
    #Now we determine for the total number of cliks
    for c in clicks:
        for e in c:
            unique_clicks.append(e)
        
    #number_of_clicks = len(Counter(unique_clicks).keys())
    number_of_clicks = len(unique_clicks)
    rho = number_of_clicks/number_of_docs
    #print(number_of_clicks,number_of_docs)
    return rho

print("In the Yandex dataset the probability that a (random) document gets clicked is:", rcm(answers,clicks))

In the Yandex dataset the probability that a (random) document gets clicked is: 0.13445559411047547


In [66]:
#Dynamic Bayesian network model
#first we will look at sigma, as we can derive this by MLE directly. We 
from operator import itemgetter
import itertools
query_clicks = {}
query_sigma = {}
for q in np.unique(query_ids)[:100]:
    indices = [i for i, j in enumerate(query_ids) if j == q]
    query_clicks[q] = list(itemgetter(*indices)(clicks))
    last_clicks = []
    for i in query_clicks[q]:
        if isinstance(i, int):
            last_clicks.append(query_clicks[q][-1])
            break
        elif len(i) > 0:
            last_clicks.append(i[-1])
        else:
            last_clicks.append(0)
    last_counter = Counter(last_clicks)
    del last_counter[0]
    try:
        click_counter = Counter(list(itertools.chain(*query_clicks[q])))
    except:
        click_counter = Counter(query_clicks[q])
    sigma_dict = dict(Counter({k:last_counter[k]/click_counter[k] for k in click_counter}))
    query_sigma[q] = sigma_dict
    

Now we have the sigma's for the DBNM, however we cannot use these sigma's to evaluate e/p as we cannot link queries/documents. To be able to use the calculated sigma's, we will devide document query combinations in 3 parts: Not relevant, relevant and highly relevant (33% of query-document pairs each) and then we calculate the average sigma per group to use in the simulation.

In [83]:
sigmas = []
for key in query_sigma:
    [sigmas.append(i) for i in list(query_sigma[key].values())]
sigmas = np.sort(sigmas)
pairs = len(sigmas)

sigma_dict = {'N': np.mean(sigmas[:int(pairs/3)]), 'R': np.mean(sigmas[int(pairs/3):-int(pairs/3)]), 'HR': np.mean(sigmas[-int(pairs/3):])}
print(sigma_dict)

{'R': 0.54446941082870659, 'N': 0.046705599807594686, 'HR': 0.98722901516920358}


In the simplified model, we know which items are examined (either all items or all items until the last read item). We can devide the number of clicks by the number of examines to calculate alpha.

In [81]:
#We already have query-clicks so now we must develop query-examines. 
#If clicks is empty, we have examined every document. If we clicked something, we examined until the last doc.
query_alphas = {}
for q in np.unique(query_ids)[:100]:
    examines = []
    indices = [i for i, j in enumerate(query_ids) if j == q]
    for i in indices:
        answer = answers[i]
        click = clicks[i] #we do not want to double track when someone clicks twice.
        if len(click) == 0:
            [examines.append(i) for i in answer]
        else:
            try:
                last_click = max([answer.index(i) for i in click])
                [examines.append(i) for i in answer[:last_click+1]]
            except:
                error = True
    examines_counter = Counter(examines)
    try:
        click_counter = Counter(list(itertools.chain(*query_clicks[q])))
    except:
        click_counter = Counter(query_clicks[q])
    #now we want to divide the clicks by the examines to get the attractive parameter
    query_alphas[q] = dict(Counter({k:click_counter[k]/examines_counter[k] for k in examines_counter}))

#now we do the same trick as before as we cannot link documents/queries. So we pick three categories.
alphas = []
for key in query_alphas:
    [alphas.append(i) for i in list(query_alphas[key].values())]
alphas = np.sort(alphas)
pairs = len(alphas)

alpha_dict = {'N': np.mean(alphas[:int(pairs/3)]), 'R': np.mean(alphas[int(pairs/3):-int(pairs/3)]), 'HR': np.mean(alphas[-int(pairs/3):])}
print(alpha_dict)

{'R': 0.032043953614566648, 'N': 0.0, 'HR': 0.55290135325137568}


Step 6: Simulate Interleaving Experiment (10 points)

In [111]:
#Run experiment n times with balanced interleaving and the random click model
def run_interleaving_rcm(N):
    
    # Initialize random click model first
    documents,_,clicks = read_data()
    rho = rcm(documents,clicks)
    
    A_winner = 0
    B_winner = 0
    Tie = 0
        
    for combination in combinations:
        I,I_ids,rank_A,rank_B = balanced_interleaving(combination,True)
        # The cutoff bool determines the lenth of the interleaved list
        # If set to false, the length is rank_A + rank_B - 1
        # Is set to true, the length is of rank_A
        
        # We ignore order since it is a stochastic process anyway  
        for n in range(0,N): #We decided to keep the interleaved list equal across the N trials.
            clicks = []
            for i in range(0,len(I)):
                random_variable = np.random.uniform()

                if random_variable < rho:
                    clicks.append(True)
                else:
                    clicks.append(False)

            winner = define_winner(clicks,I_ids)

            if winner == "A":
                A_winner += 1
            elif winner == "B":
                B_winner += 1
            else:
                Tie += 1
            
        
    total = A_winner + B_winner + Tie    
    print ("Percentage of cases that E outperforms P:",round(100*A_winner/total,2))
    #print ("P outperforms E",100*B_winner/total, "percent of the time")
    #print ("It is a tie",100*Tie/total, "percent of the time")

In [113]:
#Run experiment n times with balanced interleaving and the SDB click model
def run_interleaving_SDBM(sigma_dict, alpha_dict,N):
    A_winner = 0
    B_winner = 0
    Tie = 0
        
    for combination in combinations:
        I,I_ids,rank_A,rank_B = balanced_interleaving(combination,True)
        # The cutoff bool determines the lenth of the interleaved list
        # If set to false, the length is rank_A + rank_B - 1
        # Is set to true, the length is of rank_A
        
        for n in range(0,N):
            clicks = []
            satisfied = False

            for i in range(0,len(I)):
                relevance = I[i]
                random_variable = np.random.uniform()

                if random_variable < alpha_dict[relevance]: #here we check if the user is attracted
                    if len(clicks) < len(I):
                        clicks.append(True)
                    random_variable_2 = np.random.uniform()
                    if random_variable_2 < sigma_dict[relevance]: #here we look if the user was satisfied
                        while len(clicks) < len(I):
                            clicks.append(False) #if the user was satisfied, he does not click the rest of the list.
                else:
                    if len(clicks) < len(I):
                        clicks.append(False)

            winner = define_winner(clicks,I_ids)

            if winner == "A":
                A_winner += 1
            elif winner == "B":
                B_winner += 1
            else:
                Tie += 1

        
    total = A_winner + B_winner + Tie    
    print ("Percentage of caess that E outperforms P:",round(100*A_winner/total,2))

Step 7: Results and Analysis (30 points)

In [104]:
print("Offline evaluation: ")
check_performance(combinations)
print("\nOnline evaluation with random click model and balanced interleaving: ")
#Random click model has a lot of ties bacause often there are 0 clicks and we can also have both 1 or 2 clicks.
run_interleaving_rcm(5)
print("\nOnline evaluation with SDBM and balanced interleaving: ")
run_interleaving_SDBM(sigma_dict, alpha_dict,5)

Offline evaluation: 
Percentage of cases that E outperforms P with average precision measure: 0.48
Percentage of cases that E outperforms P with ERR measure: 0.5
Percentage of cases that E outperforms P with NDCG measure: 0.5
Percentage of cases that E ties with P with average precision measure: 0.05
Percentage of cases that E ties with P with ERR measure: 0.0
Percentage of cases that E ties with P with NDCG measure: 0.0

Online evaluation with random click model and balanced interleaving: 
Percentage of cases that E outperforms P: 22.18

Online evaluation with SDBM and balanced interleaving: 
Percentage of caess that E outperforms P: 32.77


In [ ]:
#Conclusion offline and online evaluation methods

In [ ]:
#Conclusion interleaving methods

In [ ]:
#Conclusion click models